# Gesture Recognition

Develop a cool feature in the smart-TV that can recognise five different gestures performed by the user which will help users control the TV without using a remote.

The gestures are continuously monitored by the webcam mounted on the TV. Each gesture corresponds to a specific command:

1. **Thumbs up:**   Increase the volume
2. **Thumbs down:** Decrease the volume
3. **Left swipe:**  'Jump' backwards 10 seconds
4. **Right swipe:** 'Jump' forward 10 seconds  
5. **Stop:**        Pause the movie

## Understanding the Dataset
1. The training data consists of a few hundred videos categorised into one of the five classes.
2. Each video (typically 2-3 seconds long) is divided into a sequence of 30 frames(images).
3. These videos have been recorded by various people performing one of the five gestures in front of a webcam - similar to what the smart TV will use. 

In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#from scipy.misc import imread, imresize
import imageio.v2 as imageio
from skimage.transform import resize
from PIL import Image
import datetime
import os
import random

In [2]:
#Code for google.colab

from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q /content/gdrive/MyDrive/Project_data.zip -d /content/Project_data/

Mounted at /content/gdrive


We set the random seed so that the results don't vary drastically.

In [3]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation.

In [4]:
#Kaggle
#train_doc = np.random.permutation(open('/kaggle/input/rnn-upload/Project_data/train.csv').readlines())
#val_doc = np.random.permutation(open('/kaggle/input/rnn-upload/Project_data/val.csv').readlines())

#Google collab
train_doc = np.random.permutation(open('/content/Project_data/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/Project_data/val.csv').readlines())

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size,img_size,img_idx):
    #print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = img_idx #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches =int(len(t)/batch_size) # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),img_size,img_size,3)) # x = len(img_idx) is the number of images you use for each video, (y,z) = (img_size,img_size) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image, (img_size,img_size))
                    batch_data[folder,idx,:,:,0] = (image[:,:,0]) - 104
                    batch_data[folder,idx,:,:,1] = (image[:,:,1]) - 117
                    batch_data[folder,idx,:,:,2] = (image[:,:,2]) - 123
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        
        # write the code for the remaining data points which are left after full batches
        leftover_batches = len(t)%batch_size
        if leftover_batches != 0:
            batch_data = np.zeros((leftover_batches,len(img_idx),img_size, img_size,3)) 
            # batch_labels is the one hot representation of the output: 10 videos with 5 columns as classes
            batch_labels = np.zeros((leftover_batches,5)) 
            for folder in range(leftover_batches): # iterate over the leftover_batches
                imgs = os.listdir(source_path +'/'+t[folder + (num_batches * batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imageio.imread(source_path +'/'+t[folder + (num_batches * batch_size)].split(';')[0] +'/'+imgs[item]).astype(np.float32)
                    image = resize(image, (img_size,img_size))
                    batch_data[folder,idx,:,:,0] = (image[:,:,0]) - 104
                    batch_data[folder,idx,:,:,1] = (image[:,:,1]) - 117
                    batch_data[folder,idx,:,:,2] = (image[:,:,2]) - 123
                        
                batch_labels[folder, int(t[folder + (num_batches * batch_size)].split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()

#Kaggle
#train_path = '/kaggle/input/rnn-upload/Project_data/train'
#val_path = '/kaggle/input/rnn-upload/Project_data/val'

#google collab
train_path = '/content/Project_data/Project_data/train'
val_path = '/content/Project_data/Project_data/val'

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

# training sequences = 663
# validation sequences = 100


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential
from keras.layers import Conv3D, BatchNormalization, Activation, MaxPooling3D, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

# Define the model
class Conv3DModel():
    def Model3D(self,frames,img_size):
      model = Sequential()

      model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(frames,img_size,img_size,3)))
      model.add(BatchNormalization())
      model.add(Activation('relu'))
      model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

      model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
      model.add(BatchNormalization())
      model.add(Activation('relu'))
      model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='same'))

      model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
      model.add(BatchNormalization())
      model.add(Activation('relu'))
      model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='same'))

      model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
      model.add(BatchNormalization())
      model.add(Activation('relu'))
      model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2), padding='same'))

      model.add(Flatten())

      model.add(Dropout(0.25))
      model.add(Dense(512, activation='relu'))
      model.add(Dropout(0.25))
      model.add(Dense(5, activation='softmax'))
      optimiser = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.7, nesterov=True)
      model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
      return model

In [8]:
#Initialising Global vars
def global_vars(img_idx,img_size,batch_size,num_epochs):
    print("Model Features: No of Epochs = {}, batch_size = {},img_size = {}, no of frames = {}".format(num_epochs,batch_size,img_size,len(img_idx)))
    return img_idx,img_size,batch_size,num_epochs

### Model 1: No of Epochs = 10 , batch_size = 12 ,shape = (120,120) , no of frames = 10


In [9]:
img_idx,img_size,batch_size,num_epochs = global_vars([0,3,6,9,12,15,18,21,24,27],120,12,10)
conv_model1=Conv3DModel()
conv_model1=conv_model1.Model3D(len(img_idx),img_size)
conv_model1.summary()

Model Features: No of Epochs = 10, batch_size = 12,img_size = 120, no of frames = 10
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 10, 120, 120, 64)  5248      
                                                                 
 batch_normalization (BatchN  (None, 10, 120, 120, 64)  256      
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 10, 120, 120, 64)  0         
                                                                 
 max_pooling3d (MaxPooling3D  (None, 5, 60, 120, 64)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 5, 60, 120, 128)   221312    
                                     

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [10]:
train_generator = generator(train_path, train_doc, batch_size,img_size,img_idx)
val_generator = generator(val_path, val_doc, batch_size,img_size,img_idx)

In [11]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', min_delta=1e-3, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [12]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print(steps_per_epoch)
print(validation_steps)

56
9


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [13]:
conv_model1.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
56/56 [==============================] - ETA: 0s - loss: 2.2269 - categorical_accuracy: 0.4027
Epoch 1: saving model to model_init_2023-04-1716_33_09.254485/model-00001-2.22694-0.40271-2.43872-0.23000.h5
56/56 [==============================] - 86s 1s/step - loss: 2.2269 - categorical_accuracy: 0.4027 - val_loss: 2.4387 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/10
56/56 [==============================] - ETA: 0s - loss: 1.0933 - categorical_accuracy: 0.5641
Epoch 2: saving model to model_init_2023-04-1716_33_09.254485/model-00002-1.09330-0.56410-1.49417-0.40000.h5
56/56 [==============================] - 69s 1s/step - loss: 1.0933 - categorical_accuracy: 0.5641 - val_loss: 1.4942 - val_categorical_accuracy: 0.4000 - lr: 0.0010
Epoch 3/10
56/56 [==============================] - ETA: 0s - loss: 0.8459 - categorical_accuracy: 0.6440
Epoch 3: saving model to model_init_2023-04-1716_33_09.254485/model-00003-0.84587-0.64404-1.00986-0.61000.h5
56/56 [================

### Model 2: No of Epochs = 15 , batch_size = 16 ,shape = (100,100) , no of frames = 12


In [14]:
img_idx,img_size,batch_size,num_epochs = global_vars([0,3,6,9,11,13,15,18,20,22,24,27],100,16,15)
conv_model2=Conv3DModel()
conv_model2=conv_model2.Model3D(len(img_idx),img_size)
conv_model2.summary()

Model Features: No of Epochs = 15, batch_size = 16,img_size = 100, no of frames = 12
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_4 (Conv3D)           (None, 12, 100, 100, 64)  5248      
                                                                 
 batch_normalization_4 (Batc  (None, 12, 100, 100, 64)  256      
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (None, 12, 100, 100, 64)  0         
                                                                 
 max_pooling3d_4 (MaxPooling  (None, 6, 50, 100, 64)   0         
 3D)                                                             
                                                                 
 conv3d_5 (Conv3D)           (None, 6, 50, 100, 128)   221312    
                                   

In [15]:
train_generator = generator(train_path, train_doc, batch_size,img_size,img_idx)
val_generator = generator(val_path, val_doc, batch_size,img_size,img_idx)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print(steps_per_epoch)
print(validation_steps)

42
7


In [16]:
conv_model2.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/15
42/42 [==============================] - ETA: 0s - loss: 2.2014 - categorical_accuracy: 0.3846
Epoch 1: saving model to model_init_2023-04-1716_33_09.254485/model-00001-2.20137-0.38462-4.06161-0.23000.h5
42/42 [==============================] - 79s 2s/step - loss: 2.2014 - categorical_accuracy: 0.3846 - val_loss: 4.0616 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/15
42/42 [==============================] - ETA: 0s - loss: 0.9858 - categorical_accuracy: 0.6063
Epoch 2: saving model to model_init_2023-04-1716_33_09.254485/model-00002-0.98578-0.60633-3.49719-0.24000.h5
42/42 [==============================] - 79s 2s/step - loss: 0.9858 - categorical_accuracy: 0.6063 - val_loss: 3.4972 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/15
42/42 [==============================] - ETA: 0s - loss: 0.8683 - categorical_accuracy: 0.6652
Epoch 3: saving model to model_init_2023-04-1716_33_09.254485/model-00003-0.86834-0.66516-0.91775-0.67000.h5
42/42 [================

### Model 3: No of Epochs = 20 , batch_size = 32 ,shape = (70,70) , no of frames = 12

In [17]:
img_idx,img_size,batch_size,num_epochs = global_vars([0,3,6,9,11,13,15,18,20,22,24,27],70,32,20)
conv_model3=Conv3DModel()
conv_model3=conv_model3.Model3D(len(img_idx),img_size)
conv_model3.summary()

Model Features: No of Epochs = 20, batch_size = 32,img_size = 70, no of frames = 12
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 12, 70, 70, 64)    5248      
                                                                 
 batch_normalization_8 (Batc  (None, 12, 70, 70, 64)   256       
 hNormalization)                                                 
                                                                 
 activation_8 (Activation)   (None, 12, 70, 70, 64)    0         
                                                                 
 max_pooling3d_8 (MaxPooling  (None, 6, 35, 70, 64)    0         
 3D)                                                             
                                                                 
 conv3d_9 (Conv3D)           (None, 6, 35, 70, 128)    221312    
                                    

In [18]:
train_generator = generator(train_path, train_doc, batch_size,img_size,img_idx)
val_generator = generator(val_path, val_doc, batch_size,img_size,img_idx)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print(steps_per_epoch)
print(validation_steps)

21
4


In [19]:
conv_model3.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 1.9356 - categorical_accuracy: 0.3891
Epoch 1: saving model to model_init_2023-04-1716_33_09.254485/model-00001-1.93557-0.38914-5.54214-0.23000.h5
21/21 [==============================] - 69s 3s/step - loss: 1.9356 - categorical_accuracy: 0.3891 - val_loss: 5.5421 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 1.0892 - categorical_accuracy: 0.5641
Epoch 2: saving model to model_init_2023-04-1716_33_09.254485/model-00002-1.08915-0.56410-3.93925-0.21000.h5
21/21 [==============================] - 62s 3s/step - loss: 1.0892 - categorical_accuracy: 0.5641 - val_loss: 3.9393 - val_categorical_accuracy: 0.2100 - lr: 0.0010
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 0.8361 - categorical_accuracy: 0.6787
Epoch 3: saving model to model_init_2023-04-1716_33_09.254485/model-00003-0.83607-0.67873-3.97862-0.27000.h5
21/21 [================

### Model 4: No of Epochs = 20 , batch_size = 32 ,shape = (70,70) , no of frames = 18

In [20]:
img_idx,img_size,batch_size,num_epochs = global_vars([0,2,4,6,7,8,10,12,14,15,16,17,18,20,22,24,26,28],70,32,20)
conv_model4=Conv3DModel()
conv_model4=conv_model4.Model3D(len(img_idx),img_size)
conv_model4.summary()

Model Features: No of Epochs = 20, batch_size = 32,img_size = 70, no of frames = 18
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_12 (Conv3D)          (None, 18, 70, 70, 64)    5248      
                                                                 
 batch_normalization_12 (Bat  (None, 18, 70, 70, 64)   256       
 chNormalization)                                                
                                                                 
 activation_12 (Activation)  (None, 18, 70, 70, 64)    0         
                                                                 
 max_pooling3d_12 (MaxPoolin  (None, 9, 35, 70, 64)    0         
 g3D)                                                            
                                                                 
 conv3d_13 (Conv3D)          (None, 9, 35, 70, 128)    221312    
                                    

In [21]:
train_generator = generator(train_path, train_doc, batch_size,img_size,img_idx)
val_generator = generator(val_path, val_doc, batch_size,img_size,img_idx)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print(steps_per_epoch)
print(validation_steps)

21
4


In [22]:
conv_model4.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 2.5798 - categorical_accuracy: 0.3379
Epoch 1: saving model to model_init_2023-04-1716_33_09.254485/model-00001-2.57979-0.33786-4.08769-0.23000.h5
21/21 [==============================] - 117s 5s/step - loss: 2.5798 - categorical_accuracy: 0.3379 - val_loss: 4.0877 - val_categorical_accuracy: 0.2300 - lr: 0.0010
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 1.0416 - categorical_accuracy: 0.5867
Epoch 2: saving model to model_init_2023-04-1716_33_09.254485/model-00002-1.04165-0.58673-3.22046-0.24000.h5
21/21 [==============================] - 97s 5s/step - loss: 1.0416 - categorical_accuracy: 0.5867 - val_loss: 3.2205 - val_categorical_accuracy: 0.2400 - lr: 0.0010
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 0.8160 - categorical_accuracy: 0.6833
Epoch 3: saving model to model_init_2023-04-1716_33_09.254485/model-00003-0.81599-0.68326-2.88045-0.23000.h5
21/21 [===============

### Model 5: No of Epochs = 20 , batch_size = 64 ,shape = (50,50) , no of frames = 12

In [23]:
img_idx,img_size,batch_size,num_epochs = global_vars([0,3,6,9,11,13,15,18,20,22,24,27],50,64,20)
conv_model5=Conv3DModel()
conv_model5=conv_model5.Model3D(len(img_idx),img_size)
conv_model5.summary()

Model Features: No of Epochs = 20, batch_size = 64,img_size = 50, no of frames = 12
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_16 (Conv3D)          (None, 12, 50, 50, 64)    5248      
                                                                 
 batch_normalization_16 (Bat  (None, 12, 50, 50, 64)   256       
 chNormalization)                                                
                                                                 
 activation_16 (Activation)  (None, 12, 50, 50, 64)    0         
                                                                 
 max_pooling3d_16 (MaxPoolin  (None, 6, 25, 50, 64)    0         
 g3D)                                                            
                                                                 
 conv3d_17 (Conv3D)          (None, 6, 25, 50, 128)    221312    
                                    

In [24]:
train_generator = generator(train_path, train_doc, batch_size,img_size,img_idx)
val_generator = generator(val_path, val_doc, batch_size,img_size,img_idx)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print(steps_per_epoch)
print(validation_steps)

11
2


In [25]:
conv_model5.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
11/11 [==============================] - ETA: 0s - loss: 2.2296 - categorical_accuracy: 0.3152
Epoch 1: saving model to model_init_2023-04-1716_33_09.254485/model-00001-2.22962-0.31523-3.42792-0.28000.h5
11/11 [==============================] - 73s 7s/step - loss: 2.2296 - categorical_accuracy: 0.3152 - val_loss: 3.4279 - val_categorical_accuracy: 0.2800 - lr: 0.0010
Epoch 2/20
11/11 [==============================] - ETA: 0s - loss: 1.2073 - categorical_accuracy: 0.5143
Epoch 2: saving model to model_init_2023-04-1716_33_09.254485/model-00002-1.20733-0.51433-3.41594-0.25000.h5
11/11 [==============================] - 70s 7s/step - loss: 1.2073 - categorical_accuracy: 0.5143 - val_loss: 3.4159 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 3/20
11/11 [==============================] - ETA: 0s - loss: 1.0167 - categorical_accuracy: 0.5792
Epoch 3: saving model to model_init_2023-04-1716_33_09.254485/model-00003-1.01675-0.57919-2.87650-0.23000.h5
11/11 [================

### Model 6 : (CONV2D + GRU) no of frames are 12 , image_size = (70,70) , batch_size 64 , no of epochs = 20

In [26]:
img_idx,img_size,batch_size,num_epochs = global_vars([0,3,6,9,11,13,15,18,20,22,24,27],70,64,20)

Model Features: No of Epochs = 20, batch_size = 64,img_size = 70, no of frames = 12


In [32]:
from keras.layers.convolutional import  Conv2D, MaxPooling2D
from keras.layers import TimeDistributed,LSTM ,ConvLSTM2D, GRU

model = Sequential()    

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),input_shape=(len(img_idx),img_size,img_size,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Flatten()))

model.add(GRU(64))
model.add(Dropout(0.25))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.25))
        
model.add(Dense(5, activation='softmax'))

In [33]:
optimiser = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.7, nesterov=True)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [34]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_16 (TimeDi  (None, 12, 70, 70, 16)   448       
 stributed)                                                      
                                                                 
 time_distributed_17 (TimeDi  (None, 12, 70, 70, 16)   64        
 stributed)                                                      
                                                                 
 time_distributed_18 (TimeDi  (None, 12, 35, 35, 16)   0         
 stributed)                                                      
                                                                 
 time_distributed_19 (TimeDi  (None, 12, 35, 35, 32)   4640      
 stributed)                                                      
                                                                 
 time_distributed_20 (TimeDi  (None, 12, 35, 35, 32)  

In [35]:
train_generator = generator(train_path, train_doc, batch_size,img_size,img_idx)
val_generator = generator(val_path, val_doc, batch_size,img_size,img_idx)

if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

print(steps_per_epoch)
print(validation_steps)

11
2


In [36]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs,verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
11/11 [==============================] - ETA: 0s - loss: 1.8007 - categorical_accuracy: 0.1825
Epoch 1: saving model to model_init_2023-04-1716_33_09.254485/model-00001-1.80065-0.18250-1.69188-0.14000.h5
11/11 [==============================] - 69s 6s/step - loss: 1.8007 - categorical_accuracy: 0.1825 - val_loss: 1.6919 - val_categorical_accuracy: 0.1400 - lr: 0.0010
Epoch 2/20
11/11 [==============================] - ETA: 0s - loss: 1.6209 - categorical_accuracy: 0.2519
Epoch 2: saving model to model_init_2023-04-1716_33_09.254485/model-00002-1.62089-0.25189-1.67615-0.22000.h5
11/11 [==============================] - 66s 7s/step - loss: 1.6209 - categorical_accuracy: 0.2519 - val_loss: 1.6762 - val_categorical_accuracy: 0.2200 - lr: 0.0010
Epoch 3/20
11/11 [==============================] - ETA: 0s - loss: 1.5310 - categorical_accuracy: 0.3167
Epoch 3: saving model to model_init_2023-04-1716_33_09.254485/model-00003-1.53100-0.31674-1.66914-0.23000.h5
11/11 [================